In [ ]:
!nvidia-smi

Thu Dec 19 06:27:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip3 install -q -U fsspec==2024.10.0

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl
!pip3 install -q -U accelerate==0.34.0
!pip3 install -q -U datasets==2.21.0
!pip3 install git+https://github.com/huggingface/transformers.git
!pip install -q -U torch

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-3htdqdk5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-3htdqdk5
  Resolved https://github.com/huggingface/transformers.git to commit 9613933b022ddbf085e2c593ed4ceea4c734179a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==0.34.0
aiohappyeyeballs==2.4.4
aiohttp==3.11.10
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.19
albumentations==1.4.20
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.20.0
astropy==6.1.7
astropy-iers-data==0.2024.12.16.0.35.48
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==24.3.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bigframes==1.29.0
bigquery-magics==0.4.0
bitsandbytes==0.42.0
bleach==6.2.0
blinker==1.9.0
blis==0.7.11
blosc2==2.7.1
bokeh==3.6.2
Bottleneck==1.4.2
bqplot==0.12.43
branca==0.8.1
CacheControl==0.14.1
cachetools==5.5.0
catalogue==2.0.10
certifi==2024.12.14
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.0
chex==0.1.88
clarabel==0.9.0
click==8.1.7
cloudpathlib==0.20.0
cloudpickle==3.1.0
cmake==3.31.2
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model_id = "google/gemma-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
<bos><start_of_turn>user
what are mental health issues<end_of_turn>
<start_of_turn>model

SyntaxError: invalid syntax (<ipython-input-48-799d17d72845>, line 1)

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
    device = "cuda:0"

    prompt_template = """
    <start_of_turn>user
    Instruction for task completion is given below.
    {query}
    <end_of_turn>\n<start_of_turn>model
    """

    prompt = prompt_template.format(query=query)
    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return decoded

In [ ]:
result = get_completion(query="what are mental health issues?", model=model, tokenizer=tokenizer)
print(result)


    user
    Instruction for task completion is given below.
    what are mental health issues?
    
model
     Sure, here's a brief overview of what mental health issues are:

Mental health issues, also known as mental disorders or mental health conditions, are conditions that affect a person's thoughts, feelings, behaviors, and overall well-being. They can manifest in various ways, including:

- Feeling sad, anxious, or stressed
- Experiencing negative thoughts or feelings
- Having difficulty regulating emotions
- Having difficulty making decisions
- Having difficulty socializing with others
- Engaging in self


In [ ]:
from datasets import load_dataset
ds1 = load_dataset("ShenLab/MentalChat16K")
ds1

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 16084
    })
})

In [ ]:
ds2 = load_dataset("fadodr/mental_health_therapy")
ds2

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 8580
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 3678
    })
})

In [ ]:
from datasets import concatenate_datasets

dataset = concatenate_datasets([ds1['train'], ds2['train'], ds2['test']])

In [ ]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 28342
})

In [ ]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'

    if data_point['input']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} here are the inputs {data_point["input"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""

    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    return text


In [ ]:
#text_column_train = [generate_prompt(data_point) for data_point in dataset['train']]
#text_column_test = [generate_prompt(data_point) for data_point in dataset['test']]

In [ ]:
#dataset['train'] = dataset['train'].add_column("prompt", text_column_train)
#dataset['test'] = dataset['test'].add_column("prompt", text_column_test)

In [ ]:
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [ ]:
#dataset = concatenate_datasets([dataset['train'], dataset['test']])

In [ ]:
dataset = dataset.shuffle(seed=3241)
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 28342
})

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
print(test_data)

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 5669
})


In [ ]:
print(train_data)

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 22673
})


In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


In [ ]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
      )
    )
    (n

In [ ]:
def formatting_func(example):
    text = f"Advice: {example['output'][0]}\nProblem: {example['input'][0]}<eos>"
    return [text]
formatting_func(dataset["train"])

["Advice: I hear you're going through a difficult time and experiencing deep sadness. It can be really tough when things that used to bring you joy no longer do, and it's understandable that this situation is affecting your ability to concentrate and find enjoyment in things.\n\nFirstly, I want to acknowledge your courage for seeking help through counseling. It's a positive step towards understanding your depression better and finding ways to lift your spirits. In therapy, you can explore the underlying causes of your sadness and learn coping strategies to manage it.\n\nOne approach that may be beneficial is cognitive-behavioral therapy (CBT). CBT focuses on identifying negative thought patterns and challenging them with more balanced and realistic thoughts. Through this approach, you can uncover any unhelpful beliefs contributing to your depression and work towards developing healthier ones.\n\nAnother helpful technique is building a strong support system around you. Surrounding yours

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
      lora_module_names.remove('lm_head')
  return list(lora_module_names)


In [ ]:
modules = find_all_linear_names(model)
print(modules)


['gate_proj', 'o_proj', 'down_proj', 'up_proj', 'v_proj', 'k_proj', 'q_proj']


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    #target_modules=modules,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")


Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


In [ ]:
import transformers
from trl import SFTTrainer



trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=3,
        max_steps=15,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)
trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


TypeError: GemmaModel.forward() got an unexpected keyword argument 'num_items_in_batch'

In [ ]:
text = "Input: I feel like im not meant to be here. Like nobody would care if I existed or not. Due to that, I've been feeling like ending my life."
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=600)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Input: I feel like im not meant to be here. Like nobody would care if I existed or not. Due to that, I've been feeling like ending my life.The input is very concerning and I would like to offer some support. It's important to know that you are not alone and that there are people who care about you. Please reach out for help if you are feeling suicidal. There are many resources available to you, including hotlines, mental health professionals, and support groups. Please don't hesitate to reach out for help.


In [ ]:
trainer.save_model("gemma_psychology_saved")